In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from tqdm import tqdm
import numpy as np
from copy import deepcopy

import rospy
import actionlib

from tams_pr2_guzheng.paths import RuckigPath
from tams_pr2_guzheng.utils import *
from tams_pr2_guzheng.msg import RunEpisodeAction


In [ ]:
rospy.init_node('jupyter', anonymous=True)

In [ ]:
run_episode = actionlib.SimpleActionClient('run_episode', RunEpisodeAction)
run_episode.wait_for_server()

In [ ]:
if os.path.exists('/tmp/plucks.json'):
    plucks = pd.read_json('/tmp/plucks.json')
else:
    plucks = pd.DataFrame(columns=[])

In [ ]:
p = RuckigPath.random(note= 'b3', string_position=.17)
p.dataframe.plot(x='y', y='z')

In [ ]:
result = run_params(run_episode, p, finger= 'ff')
row = row_from_result(result)
row['params'] = result.parameters
row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
plucks = pd.concat((plucks, row_df), ignore_index=True)

In [ ]:
original_p = deepcopy(p)

In [ ]:
p = deepcopy(original_p)

In [ ]:
plucks_along_string = pd.DataFrame(columns=[])

In [ ]:
for n in tqdm(np.repeat(np.arange(0.02, string_length(p.note), 0.02), 3)):
    print(n)
    p.string_position = n
    result = run_params(run_episode, p, finger= 'ff')
    row = row_from_result(result)
    print(f"loudness: {row['loudness']}")
    row['params'] = result.parameters
    row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
    plucks_along_string = pd.concat((plucks_along_string, row_df), ignore_index=True)

In [ ]:
f=plt.figure()
plucks_along_string.plot(kind='scatter', x='string_position', y='loudness', ax=f.gca())
f.savefig('/tmp/plucks_along_string.svg')

In [ ]:
plucks_along_string.to_json('/tmp/plucks_along_string.json')